# Adicionando funções externas utilizando LangChain

LangChain é um framework para criação de aplicações de IA e naturalmente ele possui ferramentas para facilitar a criação de funções externas para serem passadas a api da OpenAI. Para a utilização do LangChain, será necessário entendermos brevemente uma outra biblioteca de Python chamada pydantic, uma bilioteca para validação de dados que facilita a construção de estruturas de dados mais robustas.

## Introduzindo pydantic

### Como criamos uma estrutura nova sem pydantic

Sem utilizar pydantic, podemos criar uma estrutura de dados utilizando funções da seguinte forma:

In [1]:
class Pessoa:
    def __init__(self, nome: str, idade: int, peso: float) -> None:
        self.nome = nome
        self.idade = idade
        self.peso = peso

Neste caso, criamos uma função que representa uma pessoa e tem os seguintes atributos: nome, idade e peso.

In [2]:
adriano = Pessoa('Adriano', 32, 68)
adriano

In [5]:
adriano.idade

32

Podemos setar os atributos com qualquer tipo de dado, pois o Python por padrão não faz checagem de tipos.

In [6]:
adriano = Pessoa('Adriano', 32, 'ashdbadgvuya')
adriano

In [7]:
adriano.peso

'ashdbadgvuya'

### Como criamos uma estrutura nova usando pydantic

A sintaxe de pydantic acaba sendo bem mais simples para a criação de classes de dados, ao compararmos com a criação de classes comuns de Python. Nela, temos que cuidar com a definição do tipo de cada atributo, pois eles serão utilizados para validar se os dados fornecidos estão corretos.

In [8]:
from pydantic import BaseModel

class pydPessoa(BaseModel):
    nome: str
    idade: int
    peso: float

In [9]:
adriano = pydPessoa(nome='Adriano', idade=32, peso=68)
adriano

pydPessoa(nome='Adriano', idade=32, peso=68.0)

In [10]:
adriano.nome

'Adriano'

O interessante é vermos que pydantic fornece uma validação automática de dados. Isso garante uma integridade muito maior em aplicações mais complexas.

In [11]:
adriano = pydPessoa(nome='Adriano', idade=32, peso='asuhdauishg')


ValidationError: 1 validation error for pydPessoa
peso
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='asuhdauishg', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/float_parsing

In [13]:
adriano = pydPessoa(nome='Adriano', idade=32, peso='68')
adriano.peso

68.0

Podemos fazer um nesting de classes de pydantic, onde uma classe de dados recebe como input outra classe de pydantic.

In [15]:
from typing import List

class pydAsimoTeam(BaseModel):
    funcionarios: List[pydPessoa]

pydAsimoTeam(funcionarios=[pydPessoa(nome='Adriano', idade=32, peso=68)])

pydAsimoTeam(funcionarios=[pydPessoa(nome='Adriano', idade=32, peso=68.0)])

E a validação continua fucnionando mesmo com esta estrutura de nesting.

In [16]:
pydAsimoTeam(funcionarios=[Pessoa(nome='Adriano', idade=32, peso=68)])

ValidationError: 1 validation error for pydAsimoTeam
funcionarios.0
  Input should be a valid dictionary or instance of pydPessoa [type=model_type, input_value=<__main__.Pessoa object at 0x000002A0918F1250>, input_type=Pessoa]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type

## Utilizando pydantic para criação de tools da OpenAI

In [ ]:
import json

def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )
    
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]


In [18]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
    celsius = 'celsius'
    fahrenheit = 'fahrenheit'

class ObterTemperaturaAtual(BaseModel):
    """Obtém a temperatura atual de uma determinada localidade"""
    local: str = Field(description='O nome da cidade', examples=['São Paulo', 'Porto Alegre'])
    unidade: Optional[UnidadeEnum]

In [19]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_temperatura = convert_to_openai_function(ObterTemperaturaAtual)
tool_temperatura

{'name': 'ObterTemperaturaAtual',
 'description': 'Obtém a temperatura atual de uma determinada localidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'description': 'An enumeration.',
    'enum': ['celsius', 'fahrenheit'],
    'type': 'string'}},
  'required': ['local']}}

## Adicionando função externa utilizando LangChain 

Agora que já sabemos criar funções que os modelos de llm entendam, podemos passar essas funções para os modelos de linguagem através da biblioteca langchain. Para isso temos duas formas, podemos utilizar o parâmetro functions ao chamar o método invoke dos chat_models:

In [20]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

resposta = chat.invoke('Qual é a temperatura de Porto Alegre', functions=[tool_temperatura])
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 105, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-3f10a6b4-3a36-4b74-b8fa-3a93f5d7a8c9-0')

Ou podemos dar um bind e criar um novo componente de chat_model que terá acesso a função sempre que for chamado o invoke. Nestes dois casos, o modelo se comportará com o parâmetro "auto" de chamamento de função, ou seja, ele chamará a função quando necessitar, caso contrário se comportará como um modelo de linguagem normal.

In [21]:
chat = ChatOpenAI()
chat_com_func = chat.bind(functions=[tool_temperatura])
resposta = chat_com_func.invoke('Qual é a temperatura de Porto Alegre')
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 105, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c08b20b3-ec81-4302-8bcb-d28d71de2f41-0')

Podemos obrigar o modelo a sempre chamar uma função da seguinte forma:

In [22]:
resposta = chat.invoke(
    'Qual é a temperatura de Porto Alegre', 
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
    )
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 119, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-19b720a6-e99c-4958-b03f-8eaef69d7452-0')

In [23]:
resposta = chat.invoke(
    'Olá', 
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
    )
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 112, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2054b9dc-8063-48d8-81f1-a383a379f004-0')

### Adicionando a uma chain

Podemos adicionar agora este modelo com funções a um prompt e criar uma chain.

In [24]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])

chain = prompt | chat.bind(functions=[tool_temperatura])

In [25]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 110, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-52bab6ca-5c75-46e0-b340-67578d02fd45-0')

In [26]:
chain.invoke({'input': 'Qual a temperatura em Floripa?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Florianopolis"}', 'name': 'ObterTemperaturaAtual'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 115, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f2a120e6-23b4-4faf-8d10-96586d3d6a98-0')